In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
os.chdir('gdrive/MyDrive/boddam-masters-project')

In [3]:
!sudo apt-get install openslide-tools
!sudo apt-get install python-openslide
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 4 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 1s (106 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/

In [4]:
import pandas as pd
from PIL import Image
import glob
import numpy as np
from openslide import ImageSlide, open_slide
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from numpy import asarray
from numpy import savez_compressed

In [7]:
train_data_dir = "data/train"
test_data_dir = "data/test"
data_width, data_height = 1000, 1000
tile_size = 256

In [12]:
train_image_data = []
train_class_data = []
categories = {"normal", "tumor"}
for label in categories:
  imgdir = train_data_dir + "/" + label
  basename = '*.svs'
  filelist = glob.glob(os.path.join(imgdir, basename))
  for filename in filelist:
    #print(f'name is {filename}')
    im = open_slide(filename)
    #im = im.convert('RGB')
    imgwidth, imgheight = im.level_dimensions[0]
    #print(('Image size is: %d x %d ' % (imgwidth, imgheight)))
    #print('Image size is: ', im.level_dimensions[0])
    #print('Image levels and downsamples are: ', im.level_downsamples)
    start_h = 0
    start_w = 0
    for m in range(start_h, imgheight, data_height):
      for n in range(start_w, imgwidth, data_width):
        if (m + data_height > imgheight) or (n + data_width > imgwidth):
          break
        print("reading region width ", n, " x ", m, " from " , filename)
        piece = im.read_region((n, m), 0, (data_width, data_height))
        img = Image.new("RGB", (data_width, data_height), tile_size)
        img.paste(piece)
        img = np.array(img)
        # the number below should be 240
        if (np.mean(img) < 180):
          if img is None or img.shape != (data_width, data_height, 3):
            print('This image is bad')
          else:
            train_image_data.append(img)
            train_class_data.append([label])
            print("added image to array")

reading region width  0  x  0  from  data/train/tumor/tumor1.svs
reading region width  1000  x  0  from  data/train/tumor/tumor1.svs
reading region width  2000  x  0  from  data/train/tumor/tumor1.svs
reading region width  3000  x  0  from  data/train/tumor/tumor1.svs
reading region width  4000  x  0  from  data/train/tumor/tumor1.svs
reading region width  5000  x  0  from  data/train/tumor/tumor1.svs
reading region width  6000  x  0  from  data/train/tumor/tumor1.svs
reading region width  7000  x  0  from  data/train/tumor/tumor1.svs
reading region width  8000  x  0  from  data/train/tumor/tumor1.svs
reading region width  9000  x  0  from  data/train/tumor/tumor1.svs
reading region width  10000  x  0  from  data/train/tumor/tumor1.svs
reading region width  11000  x  0  from  data/train/tumor/tumor1.svs
reading region width  12000  x  0  from  data/train/tumor/tumor1.svs
reading region width  13000  x  0  from  data/train/tumor/tumor1.svs
reading region width  14000  x  0  from  data/t

In [14]:
print("size of image data: ", len(train_image_data))
print("size of labels: ", len(train_class_data))
print("shape of one image: ", np.array(train_image_data[20]).shape)

train_image_data = train_image_data[300:]
train_class_data = train_class_data[300:]
print("size of image data: ", len(train_image_data))
print("size of labels: ", len(train_class_data))
print("shape of one image: ", np.array(train_image_data[20]).shape)

size of image data:  631
size of labels:  631
shape of one image:  (1000, 1000, 3)
size of image data:  331
size of labels:  331
shape of one image:  (1000, 1000, 3)


In [15]:
train_image_data = np.array(train_image_data)
print(train_image_data.shape)

train_class_data = np.array(train_class_data)
le = LabelEncoder()
train_class_data = le.fit_transform(train_class_data)
train_class_data = to_categorical(train_class_data)
print(train_class_data[9])

(331, 1000, 1000, 3)
[0. 1.]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
train_image_data = asarray(train_image_data)
savez_compressed('train_image_data.npz', train_image_data)

In [17]:
train_class_data = asarray(train_class_data)
savez_compressed('train_class_data.npz', train_class_data)

In [12]:
test_image_data = []
test_class_data = []
categories = {"normal", "tumor"}
for label in categories:
  imgdir = test_data_dir + "/" + label
  basename = '*.svs'
  filelist = glob.glob(os.path.join(imgdir, basename))
  for filename in filelist:
    im = open_slide(filename)
    imgwidth, imgheight = im.level_dimensions[0]
    start_h = 0
    start_w = 0
    for m in range(start_h, imgheight, data_height):
      for n in range(start_w, imgwidth, data_width):
        if (m + data_height > imgheight) or (n + data_width > imgwidth):
          break
        print("reading region width ", n, " x ", m, " from " , filename)
        piece = im.read_region((n, m), 0, (data_width, data_height))
        img = Image.new("RGB", (data_width, data_height), tile_size)
        img.paste(piece)
        img = np.array(img)
        # the number below should be 240
        if (np.mean(img) < 180):
          if img is None or img.shape != (data_width, data_height, 3):
            print('This image is bad')
          else:
            test_image_data.append(img)
            test_class_data.append([label])
            print("added image to array")

Streaming output truncated to the last 5000 lines.
reading region width  124000  x  4000  from  data/test/tumor/tumor2.svs
reading region width  125000  x  4000  from  data/test/tumor/tumor2.svs
reading region width  126000  x  4000  from  data/test/tumor/tumor2.svs
reading region width  127000  x  4000  from  data/test/tumor/tumor2.svs
reading region width  128000  x  4000  from  data/test/tumor/tumor2.svs
reading region width  129000  x  4000  from  data/test/tumor/tumor2.svs
reading region width  130000  x  4000  from  data/test/tumor/tumor2.svs
reading region width  0  x  5000  from  data/test/tumor/tumor2.svs
reading region width  1000  x  5000  from  data/test/tumor/tumor2.svs
reading region width  2000  x  5000  from  data/test/tumor/tumor2.svs
reading region width  3000  x  5000  from  data/test/tumor/tumor2.svs
reading region width  4000  x  5000  from  data/test/tumor/tumor2.svs
reading region width  5000  x  5000  from  data/test/tumor/tumor2.svs
reading region width  6000  

In [16]:
print("size of image data: ", len(test_image_data))
print("size of labels: ", len(test_class_data))
print("shape of one image: ", np.array(test_image_data[20]).shape)
print("first image data class: ", test_class_data[80])
print("last image data class: ", test_class_data[200])

test_image_data_2 = test_image_data[80:200]
test_class_data_2 = test_class_data[80:200]
print("size of image data: ", len(test_image_data_2))
print("size of labels: ", len(test_class_data_2))
print("shape of one image: ", np.array(test_image_data_2[20]).shape)

size of image data:  1311
size of labels:  1311
shape of one image:  (1000, 1000, 3)
first image data class:  ['normal']
last image data class:  ['tumor']
size of image data:  120
size of labels:  120
shape of one image:  (1000, 1000, 3)


In [17]:
test_image_data_2 = np.array(test_image_data_2)
print(test_image_data_2.shape)

test_class_data_2 = np.array(test_class_data_2)
le = LabelEncoder()
test_class_data_2 = le.fit_transform(test_class_data_2)
test_class_data_2 = to_categorical(test_class_data_2)
print(test_class_data_2[9])

(120, 1000, 1000, 3)
[1. 0.]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
test_image_data_2 = asarray(test_image_data_2)
savez_compressed('test_image_data.npz', test_image_data_2)

In [19]:
test_class_data_2 = asarray(test_class_data_2)
savez_compressed('test_class_data.npz', test_class_data_2)